# Footfall data downloaded from the City of Melbourne's Open Data Website:  
### https://data.melbourne.vic.gov.au/Transport/Pedestrian-Counting-System-Monthly-counts-per-hour/b2ak-trbp
#### By selecting Export --> CSV
##### Check where I downloaded the sensor location data from

#### Code initialisation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from datetime import date, datetime

#### Import pedestrian count data

In [2]:
sensor_counts = pd.read_csv('Data/Pedestrian_Counting_System_-_Monthly__counts_per_hour_.csv')
sensor_locations = pd.read_csv('Data/melbourne_locations.csv')

#### Rename columns to all lowercase (to facilitate joining)

In [3]:
sensor_counts.rename({'Date_Time': 'datetime', 'Year': 'year', 'Month':'month', 'Mdate': 'mdate', 
                      'Day': 'day', 'Time': 'time', 'Sensor_ID': 'sensor_id', 'Hourly_Counts': 'hourly_counts'}, 
                     axis = 1, inplace = True)

#### Drop unneeded columns

In [4]:
sensor_counts.drop(['ID', 'Sensor_Name'], axis = 1, inplace = True)
sensor_locations.drop(['sensor_description', 'sensor_name', 'installation_date', 'status', 'note', 'direction_1',
                      'direction_2'], axis = 1, inplace = True)

#### Join two dataframes so location and count info in same place

In [5]:
location_counts = pd.merge(sensor_locations, sensor_counts, on='sensor_id', how='inner')

#### It could be useful to drop sensors that do not have as many recorded countsfor now leaving them in, but may drop in the future

#### Properly format datetime column

In [6]:
location_counts['datetime'] = pd.to_datetime(location_counts['datetime'], format = '%B %d, %Y %I:%M:%S %p')

#### Order by datetime column

In [7]:
location_counts = location_counts.sort_values(by=['datetime'])
location_counts.reset_index(inplace = True, drop = True)

#### September, 2010 dates have problems:
All dates have a timestamp of 00:00, presume they are in order of hour of day, for each day there are only 23 hours worth of data
For now, in later stages just filter out 2010 data

In [594]:
# # Get data for just one sensor
# one_sensor =  location_counts[location_counts.sensor_id == 4]
# one_year = one_sensor[one_sensor.year==2010]
# # Set the datetime as the index
# one_sensor.set_index('datetime', inplace = True)
# duplicates = one_sensor[one_sensor.index.duplicated()]
# for day in duplicates.mdate.unique():
#     one_day = duplicates[duplicates.mdate == day]
#     print(day, len(one_day))

#### List the number of missing hours in each year of data for each sensor

In [241]:
def check_sensor_data_completeness(sensor_number):
    
    # Get data for just one sensor
    one_sensor =  location_counts[location_counts.sensor_id == sensor_number]

    # Go for only years after 2010 where messed up data is
    one_sensor = one_sensor[one_sensor.year >2010]
    one_sensor = one_sensor.sort_values(by=['datetime'])
    
    # Save
    yearly_missing_vals = {}
    
    # Loop through each year, find the number of hourly values there are and compare this against
    # the number of values there should be in that year (calculated using daterange and the first and last
    # hour in the year)
    for year in np.sort(one_sensor.year.unique()):
        one_year = one_sensor[one_sensor.year==year]
        vals_this_year = len(one_year)
        expected_vals_this_year = len(pd.date_range(date(year, 1, 1), datetime(year, 12, 31, 23), freq = 'H'))
        yearly_missing_vals[year] = expected_vals_this_year - vals_this_year
    
    # Convert to dataframe
    yearly_missing_vals_df = pd.DataFrame.from_dict(yearly_missing_vals, orient='index', columns = [sensor_number])
    
    return yearly_missing_vals_df

def count_number_missing_blocks(sensor_number):
    this_sensor = []
    for year in location_counts.year.unique()[2:]:
        # Get data for just one sensor, in just one year (where there was just one missing value)
        one_sensor = location_counts[location_counts.sensor_id == sensor_number].copy()
        one_sensor_one_yr = one_sensor[one_sensor.year == year].copy()
        one_sensor_one_yr.reset_index(inplace=True)

        if len(one_sensor_one_yr) == 0:
            this_sensor.append(np.nan)
        else:
            # Create a new column listing the timedifference between each row and the previous row
            one_sensor_one_yr['timediff'] = one_sensor_one_yr['datetime'].diff().apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')

            # Set this value for the first row using a timestamp for the first Jan
            first_jan = datetime(year = year, month = 1, day=1, hour=0, minute=0, second=0)
            # If the first row is the first of Jan, then set the timediff to be 60 so this doesnt flag as a mising block of data
            if one_sensor_one_yr['datetime'].loc[0] == first_jan:
                one_sensor_one_yr.at[one_sensor_one_yr.index[0], 'timediff'] = 60
            # If it's not the fist_jan, then set it to how many hours are between that timestamp and the first Jan
            else:
                one_sensor_one_yr.at[one_sensor_one_yr.index[0], 'timediff'] = (one_sensor_one_yr['datetime'].loc[0]-first_jan).days*24                      

            # Find the number rows where the time difference with the previous row doesn't equal 60 
            number_blocks_of_rows = len(one_sensor_one_yr.loc[one_sensor_one_yr['timediff'] != 60])

            # Check the last row is the 23rd hour of 31st December
            # If its not, then add 1 to the number of missing rows of data
            last_dec = datetime(year = year, month = 12, day=31, hour=23, minute=0, second=0)  
            if one_sensor_one_yr['datetime'].loc[len(one_sensor_one_yr)-1] != last_dec:
                number_blocks_of_rows+=1

            # Add the number missing rows to the list for this sensor    
            this_sensor.append(number_blocks_of_rows)
            
    return this_sensor

In [249]:
# Inititalise dataframe to store the number of missing values in each year across all sensors
# and the number of blocks in which the missing values are located
missing_vals_per_year_all_sensors = pd.DataFrame(None)
missing_vals_block_sizes = pd.DataFrame(None)

# Get a list of sorted sensor numbers
sensor_numbers_sorted =  sorted(location_counts['sensor_id'].unique().tolist())

# Loop through each sensor
for sensor_number in sensor_numbers_sorted:
    # Return dataframes containing the number of missing values/blocks of missing vals in each year
    missing_vals_per_year_this_sensor = check_sensor_data_completeness(sensor_number)
    missing_vals_block_sizes_this_sensor =  count_number_missing_blocks(sensor_number)
    
    # Add to dataframes containing values for all sensors
    missing_vals_block_sizes[sensor_number] = missing_vals_block_sizes_this_sensor
    missing_vals_per_year_all_sensors[sensor_number] = missing_vals_per_year_this_sensor
    
    # Set index to year names
    missing_vals_block_sizes.index = location_counts.year.unique()[2:]
    missing_vals_per_year_all_sensors.index = location_counts.year.unique()[2:]

### Find the number of sensors with no data in each year

In [14]:
# missing_vals_per_year_all_sensors.isna().sum(axis = 1)

# for sensor_number in sensor_numbers_sorted:
# # missing_vals_per_year_all_sensors.loc[:,1]  
# for sensor_number in sensor_numbers_sorted:
#     if (missing_vals_per_year_all_sensors[sensor_number] == 0).all() == True:
#         print("Full data all years")

### Find sensors which don't have any years with absoloutly no data

In [75]:
missing_vals_per_year_all_sensors[missing_vals_per_year_all_sensors.columns[~missing_vals_per_year_all_sensors.isnull().any()]]

,1,2,3,4,5,6,7,8,9,10,11,12,14,17,18
2011,8472,0,24,0,0,0,0,0,0,432,0,0,0,0,0
2012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2013,1,1,1,1,1,1,409,1,1,1,1,2928,1,1,1
2014,1105,1,1,337,1,1,5664,1,1,1,1,1,1,1,1
2015,1129,1,1,1,1,1,1609,193,1,1,1,1,49,4320,1
2016,1,1,2208,2184,1,1,1369,19,4,11,608,584,1,2,5
2017,1,1,577,3649,3096,1,433,433,1,1,577,1,2041,1,1
2018,0,0,768,0,264,433,24,0,0,0,0,0,2256,1056,0
2019,144,240,802,0,0,0,2352,504,0,0,336,0,553,624,0
2020,0,0,120,0,0,0,0,312,0,0,0,0,1224,336,0


#### Check that where there are 0s in the  missing_blocks_sizes_df that there is also a 0 in the dataframe with the number of missing vals

In [258]:
# Set values over or equal to 1 in both dataframes
missing_block_or_not = missing_vals_block_sizes.copy()
missing_block_or_not[missing_block_or_not >= 1] = 1
missing_vals_or_not = missing_vals_per_year_all_sensors.copy()
missing_vals_or_not[missing_vals_or_not >= 1] = 1

# Check if they are the same
equality  = missing_block_or_not.eq(missing_vals_or_not)|(missing_block_or_not.isna()&missing_vals_or_not.isna())
print(np.all(equality))

True


In [257]:
missing_vals_per_year_all_sensors

,1,2,3,4,5,6,7,8,9,10,...,73,75,76,77,78,79,84,85,86,87
2011,8472,0,24,0,0,0,0,0,0,432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,1,1,1,1,1,1,409,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,1105,1,1,337,1,1,5664,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,1129,1,1,1,1,1,1609,193,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,1,1,2208,2184,1,1,1369,19,4,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,1,1,577,3649,3096,1,433,433,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,0,0,768,0,264,433,24,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,144,240,802,0,0,0,2352,504,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020,0,0,120,0,0,0,0,312,0,0,...,8040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Looking at trends in sensor counts
Look to see which sensors have valid records for a full calendar year at a time. Some sensors were added at later years, and some stop working at points throughout the year

# Datasets with city features and location coordinates

### clean datetime

## Dataset with building information across the city
 "Data collected as part of the City of Melbourne's Census of Land Use and Employment (CLUE). The data covers the period 2002-2018. It shows selected building attributes including location, construction year, refurbished year, number of floors above ground, predominant space use, bicycle/shower facilities and building accessibility. "

In [674]:
buildings = pd.read_csv('Data/Buildings_with_name__age__size__accessibility__and_bicycle_facilities.csv')

### Clean buildings dataset

In [675]:
# Remove columns not needed
buildings.drop(['Block ID', 'Property ID', 'Base property ID', 'Building name', 'Construction year', 'Location',
                'Street address', 'CLUE small area', 'Refurbished year', 'Has showers', 'Accessibility type description'], axis = 1, inplace = True)
# Rename columns
buildings.rename({'Census year': 'year', 'Number of floors (above ground)': 'n_floors', 
                 'Predominant space use': 'building_use', 'Accessibility type': 'access_type',
                 'Accessibility rating': 'access_rating', 'Bicycle spaces': 'bike_spaces',
                 'x coordinate': 'Longitude', 'y coordinate': 'Latitude'}, axis =1, inplace = True)

# Drop 130 buildings that don't have location coordinates
buildings.dropna(subset = ['Longitude'], axis = 0, inplace = True)

# Keep only buildings from 2010 onwards (as this is when camera data is for)
buildings = buildings[buildings['year'] >= 2010]

# Shorten variable names for access type, and convert the variable to categorical
buildings.access_type = buildings.access_type.replace({'Not determined or not applicable': np.nan, 'Low level of accessibility': 'low', 
                               'Moderate level of accessibility': 'moderate', 'High level of accessibility': 'high'})
cat_type = pd.CategoricalDtype(categories = ['low', 'moderate', 'high'], ordered = True)
buildings.access_type = buildings.access_type.astype(cat_type)

#### Check for variables with missing values
Columns which have a high proportion of NULL should perhaps be dropped

In [632]:
round(buildings.isnull().sum()/len(buildings) * 100,1)
# buildings['access_type'].value_counts(dropna = False)

year              0.0
n_floors          0.0
building_use      0.0
access_type      63.3
access_rating     8.7
bike_spaces      65.2
longitude         0.0
latitude          0.0
dtype: float64

In [636]:
#see correlations
# sns.heatmap(buildings.corr(), annot = True);

#### Save to file

In [676]:
buildings.to_csv('Cleaned_data/buildings_clean.csv', header = buildings.columns, index=False)

## Dataset with Landmarks & Places of Interest
"This dataset contains a description and co-ordinates of places of interest within the City of Melbourne.

Themes include: Community Use, Education Centre, Health Services, Leisure/Recreation, Mixed Use, Office, Place Of Assembly, Place of Worship, Purpose Built, Retail, Transport, Vacant Land

Sub-themes include: Art Gallery/Museum, Church, Function/Conference/Exhibition Centre, Informal Outdoor Facility (Park/Garden/Reserve), Major Sports & Recreation Facility, Office, Public Buildings, Public Hospital, Railway Station, Retail/Office/Carpark, Tertiary (University), Theatre Live"

In [637]:
landmarks = pd.read_csv('Data/Landmarks_and_places_of_interest__including_schools__theatres__health_services__sports_facilities__places_of_worship__galleries_and_museums..csv')

#### Cleaning landmarks dataset

In [642]:
# Check whether any of the variables have missing data
landmarks.isnull().sum()

# Split co-ordinates into latitude and longitude columns like building data is in
split_data = landmarks['Co-ordinates'].str.strip(')').str.strip('(').str.split(', ')
landmarks['latitude'] = split_data.apply(lambda x: x[0])
landmarks['longitude'] = split_data.apply(lambda x: x[1])
landmarks.drop('Co-ordinates', axis = 1, inplace = True)

# Rename columns
landmarks.rename({'Theme': 'theme', 'Sub Theme': 'sub_theme', 'Feature Name': 'feature_name'}, axis =1, inplace = True)

#### Print a summary of the kind of landmarks present in the dataset by theme and subtheme

In [644]:
# landmarks.groupby(['theme', 'sub_theme'])['sub_theme'].count()

#### Save to file

In [664]:
landmarks.to_csv('Cleaned_data/landmarks_clean.csv', header = landmarks.columns, index=False)

## Dataset with Bike Share Locations and capacity
### Even though this program ended in 2019, the docks were in use for the majority of years that this analysis will use
"This dataset show the historical Melbourne Bike Share docks. This program came to an end in November 2019.

Contains the bike share dock locations that were deployed across Melbourne as part of the Melbourne Bike Share Program.

Melbourne Bike Share is a joint RACV/Victorian Government bicycle hire scheme. It allows commuters to hire a bike from a dock location and return it to another dock location in the city. This dataset contains the bike share dock locations and capacity across the city."

In [645]:
bikes = pd.read_csv('Data/Bike_Share_Dock_Locations.csv')

#### Clean bike share data

In [646]:
# Check for variables with missing values
bikes.isnull().sum()
# Remove unneeded columns
bikes.drop(['rental_method', 'location', 'name'], axis =1, inplace= True)
# Rename columns
bikes.rename({'lat': 'latitude', 'lon': 'longitude'}, axis =1, inplace = True)

## City Lighting
"This dataset contains information such as location, lighting type and wattage of feature lighting across City of Melbourne.

Feature lights are usually found around high profile areas of the city."

In [648]:
lights = pd.read_csv('Data/Feature_Lighting__including_light_type__wattage_and_location_.csv')

#### Clean lighting dataset

In [649]:
# Check for variables with missing values
lights.isnull().sum()
# Remove unneeded columns
lights.drop(['asset_number', 'asset_description', 'mounting_type_lupvalue', 'location'], axis = 1, inplace = True)
# Rename columns
lights.rename({'lat': 'latitude', 'lon': 'longitude'}, axis =1, inplace = True)

#### Save to csv

In [659]:
lights.to_csv('Cleaned_data/lights_clean.csv', header = lights.columns, index=False)

## Street Infrastructure
"The City of Melbourne owns and maintains various objects and pieces of equipment installed on streets and roads for various purposes. This dataset includes Barbeques, Bicycle Rails, Bin Corrals, Bollards, Drinking Fountains, Floral Crate/Planter Boxs, Hoops, Horse Troughs, Information Pillars, Litter Bins, Picnic Setting, Seats."

In [668]:
street_inf = pd.read_csv('Data/Street_furniture_including_bollards__bicycle_rails__bins__drinking_fountains__horse_troughs__planter_boxes__seats__barbecues.csv')

#### Clean street infrastructure dataset

In [669]:
# Check for variables with missing values
street_inf.isnull().sum()

# Drop unneeded columns
street_inf.drop(['GIS_ID', 'DESCRIPTION', 'MODEL_NO', 'MODEL_DESCR', 'DIVISION', 'COMPANY',
                'LOCATION_DESC', 'EVALUATION_DATE', 'EASTING', 'NORTHING', 'UploadDate'], axis = 1, inplace = True)

# Split coordinates into lat/long coordinate columns
split_data = street_inf['CoordinateLocation'].str.strip(')').str.strip('(').str.split(', ')
street_inf['latitude'] = split_data.apply(lambda x: x[0])
street_inf['longitude'] = split_data.apply(lambda x: x[1])
street_inf.drop('CoordinateLocation', axis = 1, inplace = True)

# Rename columns to be only lowercase
street_inf.rename({'ASSET_CLASS': 'asset_class', 'ASSET_TYPE': 'feature', 'CONDITION_RATING': 'condition_rating'},
                 axis = 1, inplace = True)

# Drop the asset class as the only value present is street furniture
street_inf.drop('asset_class', axis =1, inplace = True)

In [162]:
street_inf.to_csv('Cleaned_data/street_inf_clean.csv', header = street_inf.columns, index=False)